In [1]:
from utils.structure import *
from utils.predicter import *
from utils.solver import *

In [2]:
# build structure and display
test_datas = open('./data/test_demo.txt', 'r')
test_datas = test_datas.readlines()
for test_data in test_datas:
    s = Structure(test_data)
#     print(s.data)
#     print(s.display())

In [3]:
# scan all test
s = Structure(test_datas[11])
print(s.display())

solver = BasicSolver(s)
print(solver.scan_all(method='square'))
print(solver.scan_all())
print(solver.scan_all(method='square'))

+-----+-----+-----+
|. . 6|4 . 9|. . 5|
|     |     |     |
|1 8 4|2 . .|9 6 3|
|     |     |     |
|5 . .|3 6 .|4 . .|
+-----+-----+-----+
|. . 3|. . 4|. . 9|
|     |     |     |
|7 . 8|. . .|3 4 2|
|     |     |     |
|4 . .|8 . .|5 . .|
+-----+-----+-----+
|. . .|9 4 6|. 5 1|
|     |     |     |
|9 . 5|. . 2|. 3 4|
|     |     |     |
|. 4 .|5 . .|2 9 .|
+-----+-----+-----+

+-----+-----+-----+
|    6|4   9|    5|
|     |     |     |
|1 8 4|2    |9 6 3|
|     |     |     |
|5 . .|3 6  |4    |
+-----+-----+-----+
|    3|    4|    9|
|     |     |     |
|7 . 8|  .  |3 4 2|
|     |     |     |
|4 . .|8 .  |5    |
+-----+-----+-----+
|     |9 4 6|  5 1|
|     |     |     |
|9   5|    2|  3 4|
|     |     |     |
|  4  |5    |2 9  |
+-----+-----+-----+

+-----+-----+-----+
|    6|4   9|    5|
|     |     |     |
|1 8 4|2 . .|9 6 3|
|     |     |     |
|5    |3 6  |4    |
+-----+-----+-----+
|  . 3|  . 4|    9|
|     |     |     |
|7 . 8|  . .|3 4 2|
|     |     |     |
|4    |8    |5    

In [4]:
# Square test
s = Structure(test_datas[11])
print(s.display())

solver = BasicSolver(s)
print(solver.check_square_drop('8'))
print(solver.ready)
print(solver.check_scanned_drop('8'))
print(solver.ready)
print(solver.check_square_drop('8'))
print(solver.ready)
solver.update()
print(solver.ready)
print(solver.display())

+-----+-----+-----+
|. . 6|4 . 9|. . 5|
|     |     |     |
|1 8 4|2 . .|9 6 3|
|     |     |     |
|5 . .|3 6 .|4 . .|
+-----+-----+-----+
|. . 3|. . 4|. . 9|
|     |     |     |
|7 . 8|. . .|3 4 2|
|     |     |     |
|4 . .|8 . .|5 . .|
+-----+-----+-----+
|. . .|9 4 6|. 5 1|
|     |     |     |
|9 . 5|. . 2|. 3 4|
|     |     |     |
|. 4 .|5 . .|2 9 .|
+-----+-----+-----+

+-----+-----+-----+
|    6|4 . 9|. . 5|
|     |     |     |
|1 8 4|2    |9 6 3|
|     |     |     |
|5    |3 6 .|4 . .|
+-----+-----+-----+
|    3|    4|. . 9|
|     |     |     |
|7   8|     |3 4 2|
|     |     |     |
|4    |8    |5    |
+-----+-----+-----+
|.    |9 4 6|. 5 1|
|     |     |     |
|9   5|  . 2|. 3 4|
|     |     |     |
|. 4  |5 . .|2 9 .|
+-----+-----+-----+

[0, 1, 4, 6, 7, 13, 14, 19, 20, 23, 25, 26, 27, 28, 30, 31, 33, 34, 37, 39, 40, 41, 46, 47, 49, 50, 52, 53, 54, 55, 56, 60, 64, 66, 67, 69, 72, 74, 76, 77, 80]
+-----+-----+-----+
|    6|4 . 9|. . 5|
|     |     |     |
|1 8 4|2    |9 6 3

In [5]:
solver.steps

[(4, '8'), (54, '8'), (67, '8')]

In [6]:
tmp = "A	 	 	6	4	 	9	 	 	5
B	1	8	4	2	 	 	9	6	3
C	5	 	 	3	6	 	4	 	 
D	 	 	3	 	 	4	 	 	9
E	7	 	8	 	 	 	3	4	2
F	4	 	 	8	 	 	5	 	 
G	?	×	×	9	4	6	 	5	1
H	9	×	5	 	 	2	 	3	4
I	?	4	×	5	 	 	2	9	 "

SyntaxError: EOL while scanning string literal (<ipython-input-6-2bd5296a0d22>, line 1)

In [ ]:
tmp = tmp.replace(' ', '.').replace('×', '.').replace('?','.').replace('	', '')
print(len(','.join(list(tmp))))
','.join(list(tmp))